In [1]:
# organize imports
import cv2
import imutils
import numpy as np

# global variables
bg = None

In [2]:
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average,accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [3]:
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    cnts, hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [4]:
def framing():
    # initialize weight for running average
    aWeight = 0.5
    fr = []#creating an empty list named fr
    fr_pre=[]
    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=800)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                cv2.imshow("Thesholded", thresholded)
                canned = cv2.Canny(thresholded,320,320)#For extrating edges we use Canny Edge detection method
                cv2.imshow("roi", roi)
                fr.append(roi)#Appending the read frame
                fr_pre.append(canned)#Appending the edge extracted frames

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

    # free up memory
    camera.release()
    cv2.destroyAllWindows()
    return fr_pre,fr

In [5]:
frames,ogframes = framing()#calling function framing & then extracting the images         
cv2.destroyAllWindows()

In [6]:
diff = []#creatin a list variable
for i in range(0,len(frames)-1):#defining the range
    print(frames[i],frames[i+1])#checking the frames presence
    diff.append(cv2.absdiff(frames[i],frames[i+1]))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [255 255   0 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [255 255   0 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 [  0   0   0 ...   0   0   0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[0 0 0 ... 0 0 

In [7]:
#Each image has 3 arrays namely for R(Red),G(Green) & B(Blue) color defining the color of the image, we need to calculate the mean among those & hence we call it intra-array mean
np.mean(diff)#intra-array mean can be simply calculated using this

9.866283434597388

In [8]:
mn = np.mean(diff)#This gives mean
st_d = np.std(diff)#This gives standard deviation
print(mn,st_d)#we check the mean & standard deviation

9.866283434597388 49.178844303323466


In [9]:
a = 4#Setting a random value we can modify it to any value 
ts = mn + (a * st_d)#defining the standard threshold value for the project/global threshold value
print('The threshold==>',ts)

The threshold==> 206.58166064789125


In [10]:
a_fr = []#Creating an empty list
for i in range(len(diff)):#Defining the for loop to be looped over all the frames obtained after finding the frames resulted from subtracting
    mn = np.mean(diff[i])#Calculating the mean for each frame
    st_d = np.std(diff[i])#Calculating the standard deviation for each frame
    fr_ts = mn + (4*st_d)#Finding the threshold values for each frame/image
    print(i,fr_ts)
    a_fr.append([i,fr_ts])#Appending the frame number & the threshold values

0 63.54869175497722
1 58.22290292502675
2 55.27985084259984
3 52.77142443669634
4 48.88087183129106
5 42.27734549131886
6 28.585813228583827
7 33.55500325795913
8 34.16397518562754
9 25.55123254313479
10 0.0
11 28.585813228583827
12 31.332444651790542
13 34.46450630152005
14 40.27049954680263
15 31.33244465179055
16 27.486480768834312
17 29.29618002510606
18 29.990018089181934
19 25.94963921835253
20 0.0
21 23.89235681468294
22 32.93500057267717
23 33.24641876338561
24 28.943137584361818
25 41.28583635878776
26 38.43067801261018
27 35.3510789538134
28 29.645092478407335
29 14.945412030124578
30 23.01884463737044
31 0.0
32 28.585813228583827
33 22.569605132091628
34 14.247616793188097
35 24.73548917495399
36 23.89235681468294
37 16.252210149153644
38 21.643583328560428
39 9.000018594393694
40 16.252210149153644
41 27.11030476896259
42 0.0
43 27.85766241717285
44 17.462455639097286
45 23.01884463737044
46 25.14661679203336
47 18.037483358648007
48 25.14661679203336
49 28.943137584361818


In [11]:
imp_fr = []#Creating an empty list
for i,ac_tr in(a_fr):#Defining the loop on the list obtained from above code
    if ac_tr >= ts:#Comapring the threshold values to the standard threshold/global threshold values
        print(i,ac_tr)
        imp_fr.append([i,ac_tr])#Appending the list with the imp frames based on their index & the values

87 219.61367539848825
88 233.26225540789662
89 218.4326287049005
91 212.68057340935312
92 226.42494908782197
93 221.95648216693766
94 213.08583442515916
95 207.13077223985823
96 235.46528074363005
97 246.91282038665295
98 251.3473998836093
99 245.82374764816817
100 247.08660346528703
101 238.59510800893256
102 223.31148979140266
103 257.4534329175835
104 271.3148290549058
105 278.176457217496
106 290.67816753510147
107 285.78722838593285
108 274.1285942151369
109 292.8909591243853
110 287.3048333361677
111 284.22329292812424
112 274.63329267359927
113 290.93305031818034
114 310.15814455998947
115 319.27626038615125
116 318.06274585924433
117 339.42907528856426
118 322.72320281286335
119 330.11942151409283
120 343.89464420048057
121 354.2662440379488
122 344.0143484567211
123 351.9491350031125
124 322.23806298241163
125 317.83255181107313
126 306.237381944249
127 276.94967247751435
128 260.75720281729286
129 290.67816753510147
130 285.00644799568596
131 283.2878438798025
132 279.8536674

In [12]:
key_fr = []#Creating an empty list
for i,_ in imp_fr:#Defining the loop over the list obtained from above code
    key_fr.append(ogframes[i])#This extracts the frames based on the index of frames 
    print(diff[i],i)

[[  0   0 255 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]] 87
[[  0   0 255 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]] 88
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 89
[[  0   0 255 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]] 91
[[  0   0 255 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 [  0   0 255 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]] 92
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 

In [13]:
for i in range(len(key_fr)):
    cv2.imshow('keys',key_fr[i])
    print("Frame no==>",i)
    k = cv2.waitKey(100) & 0xFF
    if k == ord('q'):
        break
cv2.destroyAllWindows()

Frame no==> 0
Frame no==> 1
Frame no==> 2
Frame no==> 3
Frame no==> 4
Frame no==> 5
Frame no==> 6
Frame no==> 7
Frame no==> 8
Frame no==> 9
Frame no==> 10
Frame no==> 11
Frame no==> 12
Frame no==> 13
Frame no==> 14
Frame no==> 15
Frame no==> 16
Frame no==> 17
Frame no==> 18
Frame no==> 19
Frame no==> 20
Frame no==> 21
Frame no==> 22
Frame no==> 23
Frame no==> 24
Frame no==> 25
Frame no==> 26
Frame no==> 27
Frame no==> 28
Frame no==> 29
Frame no==> 30
Frame no==> 31
Frame no==> 32
Frame no==> 33
Frame no==> 34
Frame no==> 35
Frame no==> 36
Frame no==> 37
Frame no==> 38
Frame no==> 39
Frame no==> 40
Frame no==> 41
Frame no==> 42
Frame no==> 43
Frame no==> 44
Frame no==> 45
Frame no==> 46
Frame no==> 47
Frame no==> 48
Frame no==> 49
Frame no==> 50
Frame no==> 51
Frame no==> 52
Frame no==> 53
Frame no==> 54
Frame no==> 55
Frame no==> 56
Frame no==> 57
Frame no==> 58
Frame no==> 59
Frame no==> 60
Frame no==> 61


In [14]:
print(len(frames))
print(len(key_fr))

334
160


In [15]:
#This function helps in writing the key-frames to a folder
for i in range(len(key_fr)):
    cv2.imshow('diff',frames[i])
    cv2.imshow('keys',key_fr[i])
    str_pos = 'keyframes2\ '+str(i)+'.jpg'
    print(str_pos)
    cv2.imwrite(str_pos,key_fr[i])#Mention preferred path
    k = cv2.waitKey(100) & 0xFF
    if k == ord('q'):
        break
cv2.destroyAllWindows()

keyframes2\ 0.jpg
keyframes2\ 1.jpg
keyframes2\ 2.jpg
keyframes2\ 3.jpg
keyframes2\ 4.jpg
keyframes2\ 5.jpg
keyframes2\ 6.jpg
keyframes2\ 7.jpg
keyframes2\ 8.jpg
keyframes2\ 9.jpg
keyframes2\ 10.jpg
keyframes2\ 11.jpg
keyframes2\ 12.jpg
keyframes2\ 13.jpg
keyframes2\ 14.jpg
keyframes2\ 15.jpg
keyframes2\ 16.jpg
keyframes2\ 17.jpg
keyframes2\ 18.jpg
keyframes2\ 19.jpg
keyframes2\ 20.jpg
keyframes2\ 21.jpg
keyframes2\ 22.jpg
keyframes2\ 23.jpg
keyframes2\ 24.jpg
keyframes2\ 25.jpg
keyframes2\ 26.jpg
keyframes2\ 27.jpg
keyframes2\ 28.jpg
keyframes2\ 29.jpg
keyframes2\ 30.jpg
keyframes2\ 31.jpg
keyframes2\ 32.jpg
keyframes2\ 33.jpg
keyframes2\ 34.jpg
keyframes2\ 35.jpg
keyframes2\ 36.jpg
keyframes2\ 37.jpg
keyframes2\ 38.jpg
keyframes2\ 39.jpg
keyframes2\ 40.jpg
keyframes2\ 41.jpg
keyframes2\ 42.jpg
keyframes2\ 43.jpg
keyframes2\ 44.jpg
keyframes2\ 45.jpg
keyframes2\ 46.jpg
keyframes2\ 47.jpg
keyframes2\ 48.jpg
keyframes2\ 49.jpg
keyframes2\ 50.jpg
keyframes2\ 51.jpg
keyframes2\ 52.jpg
key